# A list of "easy" linear systems

Consider $\mathbf{A} \mathbf{x} = \mathbf{b}$, $\mathbf{A} \in \mathbb{R}^{n \times n}$. Or, consider matrix inverse (if you want). $\mathbf{A}$ can be huge. Keep massive data in mind: 1000 Genome Project, NetFlix, Google PageRank, finance, spatial statistics, ... We should be alert to many easy linear systems. 

Don't blindly use `A \ b` and `inv` in Julia or `solve` function in R. **Don't waste computing resources by bad choices of algorithms!**

## Diagonal matrix

Diagonal $\mathbf{A}$: $n$ flops. Use `Diagonal` type of Julia.

In [1]:
using BenchmarkTools

srand(280)
n = 1000
A = diagm(randn(n)) # a full matrix
b = randn(n)

@which A \ b

\(A::AbstractArray{T,2} where T, B::Union{AbstractArray{T,1}, AbstractArray{T,2}} where T) in Base.LinAlg at linalg/generic.jl:820

In [2]:
# check `istril(A)` and `istriu(A)`, then call `Diagonal(A) \ b`
@benchmark A \ b

BenchmarkTools.Trial: 
  memory estimate:  15.95 KiB
  allocs estimate:  5
  --------------
  minimum time:     739.008 μs (0.00% GC)
  median time:      931.755 μs (0.00% GC)
  mean time:        1.127 ms (0.07% GC)
  maximum time:     4.613 ms (0.00% GC)
  --------------
  samples:          4408
  evals/sample:     1

In [3]:
# O(n) computation, no extra array allocation
@benchmark Diagonal(A) \ b

BenchmarkTools.Trial: 
  memory estimate:  15.95 KiB
  allocs estimate:  5
  --------------
  minimum time:     2.158 μs (0.00% GC)
  median time:      3.088 μs (0.00% GC)
  mean time:        4.701 μs (22.05% GC)
  maximum time:     304.640 μs (96.55% GC)
  --------------
  samples:          10000
  evals/sample:     9

## Bidiagonal, tridiagonal, and banded matrices

Bidiagonal, tridiagonal, or banded $\mathbf{A}$: Band LU, band Cholesky, ... roughly $O(n)$ flops.   
* Use [`Bidiagonal`](https://docs.julialang.org/en/stable/stdlib/linalg/#Base.LinAlg.Bidiagonal), [`Tridiagonal`](https://docs.julialang.org/en/stable/stdlib/linalg/#Base.LinAlg.Tridiagonal), [`SymTridiagonal`](https://docs.julialang.org/en/stable/stdlib/linalg/#Base.LinAlg.SymTridiagonal) types of Julia.

In [4]:
srand(280) 

n  = 1000
dv = randn(n)
ev = randn(n - 1)
b  = randn(n)
# symmetric tridiagonal matrix
A  = SymTridiagonal(dv, ev)

1000×1000 SymTridiagonal{Float64}:
 0.126238   0.618244    ⋅        …    ⋅          ⋅          ⋅      
 0.618244  -2.34688    1.10206        ⋅          ⋅          ⋅      
  ⋅         1.10206    1.91661        ⋅          ⋅          ⋅      
  ⋅          ⋅        -0.447244       ⋅          ⋅          ⋅      
  ⋅          ⋅          ⋅             ⋅          ⋅          ⋅      
  ⋅          ⋅          ⋅        …    ⋅          ⋅          ⋅      
  ⋅          ⋅          ⋅             ⋅          ⋅          ⋅      
  ⋅          ⋅          ⋅             ⋅          ⋅          ⋅      
  ⋅          ⋅          ⋅             ⋅          ⋅          ⋅      
  ⋅          ⋅          ⋅             ⋅          ⋅          ⋅      
  ⋅          ⋅          ⋅        …    ⋅          ⋅          ⋅      
  ⋅          ⋅          ⋅             ⋅          ⋅          ⋅      
  ⋅          ⋅          ⋅             ⋅          ⋅          ⋅      
 ⋮                               ⋱                                 
  ⋅          

In [5]:
# convert to a full matrix
Afull = full(A)

# LU decomposition (2/3) n^3 flops!
@benchmark Afull \ b

BenchmarkTools.Trial: 
  memory estimate:  7.65 MiB
  allocs estimate:  8
  --------------
  minimum time:     12.811 ms (0.00% GC)
  median time:      14.951 ms (0.00% GC)
  mean time:        16.320 ms (3.43% GC)
  maximum time:     62.748 ms (0.00% GC)
  --------------
  samples:          306
  evals/sample:     1

In [6]:
# specialized algorithm for tridiagonal matrix, O(n) flops
@benchmark A \ b

BenchmarkTools.Trial: 
  memory estimate:  23.89 KiB
  allocs estimate:  6
  --------------
  minimum time:     13.183 μs (0.00% GC)
  median time:      15.355 μs (0.00% GC)
  mean time:        17.816 μs (7.68% GC)
  maximum time:     1.776 ms (97.12% GC)
  --------------
  samples:          10000
  evals/sample:     1

## Triangular matrix

Triangular $\mathbf{A}$: $n^2$ flops to solve linear system.

In [7]:
srand(280)

n = 1000
A = tril(randn(n, n))
b = randn(n)

# check istril() then triangular solve
@benchmark A \ b

BenchmarkTools.Trial: 
  memory estimate:  7.95 KiB
  allocs estimate:  2
  --------------
  minimum time:     773.806 μs (0.00% GC)
  median time:      835.609 μs (0.00% GC)
  mean time:        861.995 μs (0.03% GC)
  maximum time:     4.149 ms (0.00% GC)
  --------------
  samples:          5772
  evals/sample:     1

In [8]:
# triangular solve directly; save the cost of istril()
@benchmark LowerTriangular(A) \ b

BenchmarkTools.Trial: 
  memory estimate:  7.97 KiB
  allocs estimate:  3
  --------------
  minimum time:     299.516 μs (0.00% GC)
  median time:      363.247 μs (0.00% GC)
  mean time:        385.947 μs (0.11% GC)
  maximum time:     2.366 ms (82.12% GC)
  --------------
  samples:          10000
  evals/sample:     1

## Block diagonal matrix

Block diagonal: Suppose $n = \sum_i n_i$. $(\sum_i n_i)^3$ vs $\sum_i n_i^3$.  

Julia has a [`blkdiag`](https://docs.julialang.org/en/stable/stdlib/linalg/?highlight=blkdiag#Base.blkdiag) function that generates a **full** matrix. Anyone interested writing a `BlockDiagonal.jl` package?

In [9]:
srand(280)

B  = 10 # number of blocks
ni = 100
A  = blkdiag([sprandn(ni, ni, 0.01) for b in 1:B]...)

1000×1000 SparseMatrixCSC{Float64,Int64} with 969 stored entries:
  [31  ,    1]  =  2.07834
  [53  ,    1]  =  -1.11883
  [58  ,    1]  =  -0.66448
  [14  ,    4]  =  1.11793
  [96  ,    5]  =  1.22813
  [81  ,    8]  =  -0.919643
  [48  ,    9]  =  1.0185
  [49  ,    9]  =  -0.996332
  [15  ,   10]  =  1.30841
  [28  ,   10]  =  -0.818757
  ⋮
  [956 ,  987]  =  -0.900804
  [967 ,  987]  =  -0.438788
  [971 ,  991]  =  0.176756
  [929 ,  992]  =  -1.17384
  [974 ,  993]  =  1.59235
  [967 ,  994]  =  0.542169
  [994 ,  995]  =  0.627832
  [998 ,  997]  =  0.60382
  [935 ,  998]  =  0.342675
  [947 ,  998]  =  0.482228
  [975 , 1000]  =  0.991598

## Kronecker product

Use
$$
\begin{eqnarray*}
    (\mathbf{A} \otimes \mathbf{B})^{-1} &=& \mathbf{A}^{-1} \otimes \mathbf{B}^{-1} \\
    (\mathbf{C}^T \otimes \mathbf{A}) \text{vec}(\mathbf{B}) &=& \text{vec}(\mathbf{A} \mathbf{B} \mathbf{C}).
\end{eqnarray*}    
$$
to avoid forming and doing computation on the potentially huge matrix $\mathbf{A} \otimes \mathbf{B}$.

## Sparse matrix

Sparsity: sparse matrix decomposition or iterative method.  
* The easiest recognizable structure. Familiarize yourself with the sparse matrix computation tools in Julia, Matlab, R (`Matrix` package), MKL (sparse BLAS), ... as much as possible.

In [10]:
srand(280)

n = 1000
# a sparse pd matrix, about 0.5% non-zero entries
A = sprand(n, n, 0.002)
A = A + A' + (2n) * I
b = randn(n)
Afull = full(A)
countnz(A) / length(A) # sparsity

0.005096

In [11]:
# dense matrix-vector multiplication
@benchmark Afull * b

BenchmarkTools.Trial: 
  memory estimate:  7.94 KiB
  allocs estimate:  1
  --------------
  minimum time:     105.706 μs (0.00% GC)
  median time:      125.913 μs (0.00% GC)
  mean time:        128.057 μs (0.00% GC)
  maximum time:     901.994 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [12]:
# sparse matrix-vector multiplication
@benchmark A * b

BenchmarkTools.Trial: 
  memory estimate:  7.94 KiB
  allocs estimate:  1
  --------------
  minimum time:     6.833 μs (0.00% GC)
  median time:      7.521 μs (0.00% GC)
  mean time:        8.325 μs (2.25% GC)
  maximum time:     385.589 μs (94.62% GC)
  --------------
  samples:          10000
  evals/sample:     5

In [13]:
# dense Cholesky decomposition
@benchmark cholfact(Afull)

BenchmarkTools.Trial: 
  memory estimate:  7.63 MiB
  allocs estimate:  8
  --------------
  minimum time:     9.097 ms (0.00% GC)
  median time:      10.173 ms (0.00% GC)
  mean time:        10.997 ms (7.98% GC)
  maximum time:     78.238 ms (85.51% GC)
  --------------
  samples:          454
  evals/sample:     1

In [14]:
# sparse Cholesky decomposition
@benchmark cholfact(A)

BenchmarkTools.Trial: 
  memory estimate:  1.33 MiB
  allocs estimate:  53
  --------------
  minimum time:     3.344 ms (0.00% GC)
  median time:      3.491 ms (0.00% GC)
  mean time:        3.672 ms (0.95% GC)
  maximum time:     14.893 ms (0.00% GC)
  --------------
  samples:          1359
  evals/sample:     1

In [15]:
# solve via dense Cholesky
xchol = cholfact(Afull) \ b
@benchmark cholfact(Afull) \ b

BenchmarkTools.Trial: 
  memory estimate:  7.64 MiB
  allocs estimate:  10
  --------------
  minimum time:     10.076 ms (0.00% GC)
  median time:      10.522 ms (0.00% GC)
  mean time:        10.712 ms (4.99% GC)
  maximum time:     12.816 ms (13.44% GC)
  --------------
  samples:          466
  evals/sample:     1

In [16]:
# solve via sparse Cholesky
xcholsp = cholfact(A) \ b
vecnorm(xchol - xcholsp)

4.020546956752686e-18

In [17]:
@benchmark cholfact(A) \ b

BenchmarkTools.Trial: 
  memory estimate:  1.36 MiB
  allocs estimate:  64
  --------------
  minimum time:     3.848 ms (0.00% GC)
  median time:      3.904 ms (0.00% GC)
  mean time:        4.012 ms (0.90% GC)
  maximum time:     5.691 ms (9.91% GC)
  --------------
  samples:          1245
  evals/sample:     1

In [18]:
# sparse solve via conjugate gradient
using IterativeSolvers

xcg, = cg(A, b)
vecnorm(xcg - xchol)

0.022750177100322466

In [19]:
@benchmark cg(A, b)

BenchmarkTools.Trial: 
  memory estimate:  33.30 KiB
  allocs estimate:  27
  --------------
  minimum time:     33.350 μs (0.00% GC)
  median time:      35.946 μs (0.00% GC)
  mean time:        41.878 μs (6.79% GC)
  maximum time:     2.910 ms (95.15% GC)
  --------------
  samples:          10000
  evals/sample:     1

## Easy plus low rank

Easy plus low rank: $\mathbf{U} \in \mathbb{R}^{n \times r}$, $\mathbf{V} \in \mathbb{R}^{r \times n}$, $r \ll n$. Woodbury formula
$$
	(\mathbf{A} + \mathbf{U} \mathbf{V}^T)^{-1} = \mathbf{A}^{-1} - \mathbf{A}^{-1} \mathbf{U} (\mathbf{I}_r + \mathbf{V}^T \mathbf{A}^{-1} \mathbf{U})^{-1} \mathbf{V}^T \mathbf{A}^{-1},
$$

* Keep HW2 Q2 in mind.  
* [`WoodburyMatrices.jl`](https://github.com/timholy/WoodburyMatrices.jl) package can be useful.

In [20]:
using BenchmarkTools, WoodburyMatrices

srand(280)
n = 1000
r = 5

A = Diagonal(rand(n))
B = randn(n, r)
D = Diagonal(rand(r))
b = randn(n)
# W = A + B * D * B'
W = SymWoodbury(A, B, D)
Wfull = Symmetric(full(W))

1000×1000 Symmetric{Float64,Array{Float64,2}}:
  1.8571     0.513107    0.872146   …   0.764278    -0.241331    0.54921 
  0.513107   4.57505    -0.636972      -1.86465     -1.92237    -1.72569 
  0.872146  -0.636972    4.81387        1.99357      1.99337     3.66327 
 -0.516414  -0.996711   -0.0919924      0.262832     0.612402    0.621834
  0.193686   1.68244    -0.770028      -0.723437    -1.4868     -1.32247 
  1.6567     0.0634435  -0.901968   …  -0.241872    -0.0356772  -0.39826 
  0.553996  -0.274515    2.21265        0.219437     2.20382     2.60902 
  0.402356   1.89288    -1.13032       -0.771441    -1.96862    -1.93483 
 -1.07744   -1.63881     1.78016        0.96551      1.7292      1.91326 
 -2.21617   -2.90695    -2.55971       -0.47867      0.855389   -0.933916
  1.29975    0.779828    4.12459    …   1.87358      0.737112    2.84136 
 -0.80833    1.44882     1.67581       -0.139063    -0.107873    0.818132
 -2.32469   -4.83109    -2.31796       -0.0346402    2.65564     

In [21]:
# solve via Cholesky
@benchmark cholfact(Wfull) \ b

BenchmarkTools.Trial: 
  memory estimate:  7.64 MiB
  allocs estimate:  9
  --------------
  minimum time:     8.826 ms (0.00% GC)
  median time:      9.621 ms (0.00% GC)
  mean time:        10.153 ms (5.76% GC)
  maximum time:     25.516 ms (10.87% GC)
  --------------
  samples:          492
  evals/sample:     1

In [22]:
# solve using Woodbury formula
@benchmark W \ b

BenchmarkTools.Trial: 
  memory estimate:  76.16 KiB
  allocs estimate:  26
  --------------
  minimum time:     19.691 μs (0.00% GC)
  median time:      24.759 μs (0.00% GC)
  mean time:        34.777 μs (21.82% GC)
  maximum time:     3.248 ms (97.15% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [23]:
# multiplication without using Woodbury structure
@benchmark Wfull * b

BenchmarkTools.Trial: 
  memory estimate:  7.94 KiB
  allocs estimate:  1
  --------------
  minimum time:     178.184 μs (0.00% GC)
  median time:      202.171 μs (0.00% GC)
  mean time:        207.340 μs (0.00% GC)
  maximum time:     555.101 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [24]:
# multiplication using Woodbury structure
@benchmark W * b

BenchmarkTools.Trial: 
  memory estimate:  24.06 KiB
  allocs estimate:  5
  --------------
  minimum time:     4.799 μs (0.00% GC)
  median time:      6.529 μs (0.00% GC)
  mean time:        9.382 μs (19.91% GC)
  maximum time:     490.339 μs (96.04% GC)
  --------------
  samples:          10000
  evals/sample:     6

## Easy plus border

Easy plus border: For $\mathbf{A}$ pd and $\mathbf{V}$ full row rank,
$$
	\begin{pmatrix}
	\mathbf{A} & \mathbf{V}^T \\
	\mathbf{V} & \mathbf{0}
	\end{pmatrix}^{-1} = \begin{pmatrix}
	\mathbf{A}^{-1} - \mathbf{A}^{-1} \mathbf{V}^T (\mathbf{V} \mathbf{A}^{-1} \mathbf{V}^T)^{-1} \mathbf{V} \mathbf{A}^{-1} & \mathbf{A}^{-1} \mathbf{V}^T (\mathbf{V} \mathbf{A}^{-1} \mathbf{V}^T)^{-1} \\
	(\mathbf{V} \mathbf{A}^{-1} \mathbf{V}^T)^{-1} \mathbf{V} \mathbf{A}^{-1} & - (\mathbf{V} \mathbf{A}^{-1} \mathbf{V}^T)^{-1}
	\end{pmatrix}.
$$

## Orthogonal matrix

Orthogonal $\mathbf{A}$: $n^2$ flops **at most**. Permutation matrix, Householder matrix, Jacobi matrix, ... take less.

## Toeplitz matrix

Toeplitz systems:
$$
	\mathbf{T} = \begin{pmatrix}
	r_0 & r_1 & r_2 & r_3 \\
	r_{-1} & r_0 & r_1 & r_2 \\
	r_{-2} & r_{-1} & r_0 & r_1 \\
	r_{-3} & r_{-2} & r_{-1} & r_0
	\end{pmatrix}.
$$
$\mathbf{T} \mathbf{x} = \mathbf{b}$, where $\mathbf{T}$ is pd and Toeplitz, can be solved in $O(n^2)$ flops. Durbin algorithm (Yule-Walker equation), Levinson algorithm (general $\mathbf{b}$), Trench algorithm (inverse). These matrices occur in auto-regressive models and econometrics.

* [`ToeplitzMatrices.jl`](https://github.com/JuliaMatrices/ToeplitzMatrices.jl) package can be useful.

## Circulant matrix

Circulant systems: Toeplitz matrix with wraparound
$$
	C(\mathbf{z}) = \begin{pmatrix}
	z_0 & z_4 & z_3 & z_2 & z_1 \\
	z_1 & z_0 & z_4 & z_3 & z_2 \\
	z_2 & z_1 & z_0 & z_4 & z_3 \\
	z_3 & z_2 & z_1 & z_0 & z_4 \\
	z_4 & z_3 & z_2 & z_1 & z_0
	\end{pmatrix},
$$
FFT type algorithms: DCT (discrete cosine transform) and DST (discrete sine transform).

## Vandermonde matrix

Vandermonde matrix: such as in interpolation and approximation problems
$$
	\mathbf{V}(x_0,\ldots,x_n) = \begin{pmatrix}
	1 & 1 & \cdots & 1 \\
	x_0 & x_1 & \cdots & x_n \\
	\vdots & \vdots & & \vdots \\
	x_0^n & x_1^n & \cdots & x_n^n
	\end{pmatrix}.
$$
$\mathbf{V} \mathbf{x} = \mathbf{b}$ or $\mathbf{V}^T \mathbf{x} = \mathbf{b}$ can be solved in $O(n^2)$ flops.

## Cauchy-like matrix

Cauchy-like matrices:
$$
	\Omega \mathbf{A} - \mathbf{A} \Lambda = \mathbf{R} \mathbf{S}^T,
$$
where $\Omega = \text{diag}(\omega_1,\ldots,\omega_n)$ and $\Lambda = \text{diag}(\lambda_1,\ldots, \lambda_n)$. $O(n)$ flops for LU and QR.

## Structured-rank matrix

Structured-rank problems: semiseparable matrices (LU and QR takes $O(n)$ flops), quasiseparable matrices, ...

In [25]:
versioninfo()

Julia Version 0.6.2
Commit d386e40c17 (2017-12-13 18:08 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin14.5.0)
  CPU: Intel(R) Core(TM) i7-6920HQ CPU @ 2.90GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Haswell)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.9.1 (ORCJIT, skylake)
